## 05. 감성 분석

- 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법으로 소셜 미디어, 여론 조사, 온라인 리뷰, 피드백 등 다양한 분야에 활용되고 있음
- 문서 내 텍스트가 나타내는 여러 주관적인 단어와 문맥을 기반으로 감성 수치를 계산하는 방법을 이용
- 감성 지수는 긍정 감성 지수와 부정 감성 지수로 구성, 이 지수들을 합산해 긍정 감성 또는 부정 감성을 결정

#### 비지도학습 및 지도학습
- 지도학습 : 학습 데이터와 타깃 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 다른 데이터의 감성 분석을 예측
- 비지도학습 : 'Lexicon'이라는 일종의 감성 어휘 사전을 이용, 'Lexicon'은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 있으며, 이를 이용해 문서의 긍정적, 부정적 감성 여부를 판단

### 지도학습 기반 감성 분석 실습 - IMDB 영화평
- 지도학습 기반 감성 분석 = 텍스트 기반의 이진 분류
- 영화평의 텍스트를 분석해 감성 분석 결과가 긍정 또는 부정인지를 예측하는 모델 만들기


In [36]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [20]:
# load dataset
PATH = './word2vec-nlp-tutorial/'

# header = 0 : 컬럼 이름으로 쓰이는 행 숫자, 데이터의 시작
# quoting = 3 : 큰 쌍따옴표 무시
review_df = pd.read_csv(PATH + "labeledTrainData.tsv", header=0, sep='\t', quoting=3) 
print(review_df.shape)
review_df.head()

(25000, 3)


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


- __id__ : 각 데이터의 id
- __sentiment__ : 영화평의 sentiment 결과값 (Target Label). 1은 긍정 평가, 0은 부정 평가
- __review__ : 영화평의 텍스트

In [7]:
print(review_df.review[0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [12]:
import re

# <br/> 을 공백으로 변환
review_df.review = review_df.review.str.replace('<br />',' ')

# 영어 문자열이 아닌 문자는 공백으로 변환
review_df.review = review_df.review.apply(lambda x : re.sub('[^a-zA-Z]'," ",x))

print(review_df.review[0])

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

In [31]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
# feature_df = review_df['review']
feature_df = review_df.drop(['id', 'sentiment'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

#### 예제
1. 피처 벡터화 : Count 벡터화, TF-IDF 벡터화
2. ML 분류 알고리즘 적용 : Logistic Regression Classifier
3. 예측 성능 측정 : ROC-AUC

#### cf. pipeline (pp.494)
- 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한번에 작성하여 더 직관성 ↑
- 대용량의 데이터의 피처 벡터화 결과를 별도로 저장하지 않고 스트림 기반에서 바로 머신러닝 알고리즘의 데이터로 입력할 수 있기 때문에 수행 시간을 절약
- 모든 데이터 전처리 작업과 Estimator 결합 가능 (예) 스케일링, 벡터 정규화, PCA 변환 작업과 분류, 회귀 등 Estimator를 결합

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print(accuracy_score(y_test, pred))
print(roc_auc_score(y_test, pred_probs))

0.8842666666666666
0.9500620120405556


In [42]:
pipeline = Pipeline([
    ('tfid_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2)) ),
    ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_prob = pipeline.predict_proba(X_test['review'])[:,1]

print(accuracy_score(y_test, pred))
print(roc_auc_score(y_test, pred_probs))

0.8949333333333334
0.9500620120405556


- TF-ODF 기반 피처 벡터화의 예측 성능이 조금 더 나아짐

## 비지도학습 기반 감성 분석 소개
- 'Lexicon' 기반 
- 많은 감성 분석용 데이터는 결정된 레이블 값을 가지고 있지 않음

#### Lexion 감성 어휘 사전
- 긍정 감성 또는 부정 감성의 정도를 의미하는 수치를 가지고 있으며 이를 감성 지수(Polarity score)라고 함
- 이 감성 지수는 단어의 위치나 주변 단어, 문맥, POS(Part of Speech) 등을 참고해 결정
- NLTK 패키지 : 감성 사전인 Lexion 모듈을 구현

#### WordNet
- NLP 패키지
- 단순한 영어 어휘 사진이 아닌 <U>시맨틱(semantic)</U> 분석을 제공하는 어휘 사전

__cf. 시맨틱(semantic)이란?__  
문맥상 의미, 동일한 단어나 문장이라도 다른 환경과 문맥에서는 다른게 표현되거나 이해할 수 있기 때문에  
(예) '밥 먹었어?' : 말 그대로 식사를 했는가? or 안부를 묻는 표현

- WordNet은 다양한 상황에서 같은 어휘라도 다르게 사용되는 어휘의 시맨틱 정보를 제공하며, 이를 위하 각각의 품사(명사, 동사, 형용사, 부사 등)로 구성된 개별 단어를 <U>synset(Sets of cognitive synonyms)</U> 이라는 개념을 이용해 표현

__cf. synset(Sets of cognitive synonyms)이란?__  
단순한 하나의 단어가 아니라 그 단어가 가지는 문맥, 시맨틱 정보를 제공하는 WordNet의 핵심 개념

### 대표적인 감성 사전
#### 0. WordNet :
- NLTK 패키지
- 예측성능이 그리 좋지 못하다

#### 1. SentiWordNet: 
- NLTK 패키지의 WordNet의 Synset 개념을 감성 분석에 적용
- Synset별로 3가지 감성 점수(sentiment score) - 긍정 감정/부정 감정/객관성 지수를 할당
- 객관성 지수란 단어가 감성과 관계없이 얼마나 객관적인지를 수치로 나타낸 것
- 문장별로 단어들의 긍정 감성 지수와 부정 감성 지수를 합산하여 최종 감성 지수를 계산하고 이에 기반해 문장이 긍정인지 부정인지를 결정

#### 2. VADER :
- 주로 소셜 미디어의 텍스트에 대한 감성 분석을 제공하기 위한 패키지
- 뛰어난 감성 분서 결과를 제공하고 비교적 빠른 수행 시간을 보장해 대용량 텍스트 데이터에 잘 사용됨

#### 3. Pattern : 
- 예측 성능 측면에서 주목 받는 패키지
- 파이썬 버전 3.X과 호환X, 버전 2.X에서만 동작


### SentiWordNet을 이용한 감성 분석
#### WordNet Synset과 SentiWordNet Synset 클래스의 이해

In [43]:
# NLTK 모든 데이터 세트와 패키지 다운로드
import nltk
# nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading pack

[nltk_data]    |   Unzipping corpora/senseval.zip.
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/sentiwordnet.zip.
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/sentence_polarity.zip.
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/shakespeare.zip.
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/sinica_treebank.zip.
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/smultron.zip.
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     /Users/gimdonghwa/nltk_data...
[nltk_data]    |   Unzipping corpora/st

True

### 1) WordNet

In [47]:
from nltk.corpus import wordnet as wn

term = 'present'

synsets = wn.synsets(term)

print(type(synsets))
print(len(synsets))
print(synsets)

<class 'list'>
18
[Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


- Synset('present.n.01') 에서 'present.n.01' = POS(Part of Speech, 품사) 태그
- 'present.n.01'에서 present는 의미, n은 명사 품사, 01은 present가 명사로 가지는 의미가 여러가지 있어서 이를 구분하는 인덱스
- Synset은 `POS`(품사), `definition`(정의), `Lemma`(동의어) 등으로 시맨틱적인 요소를 표현할 수 있음

In [49]:
print(dir(synsets[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_all_hypernyms', '_definition', '_examples', '_frame_ids', '_hypernyms', '_instance_hypernyms', '_iter_hypernym_lists', '_lemma_names', '_lemma_pointers', '_lemmas', '_lexname', '_max_depth', '_min_depth', '_name', '_needs_root', '_offset', '_pointers', '_pos', '_related', '_shortest_hypernym_paths', '_wordnet_corpus_reader', 'also_sees', 'attributes', 'causes', 'closure', 'common_hypernyms', 'definition', 'entailments', 'examples', 'frame_ids', 'hypernym_distances', 'hypernym_paths', 'hypernyms', 'hyponyms', 'in_region_domains', 'in_topic_domains', 'in_usage_domains', 'instance_hypernyms', 'instance_hyponyms', 'jcn_similarity', 'lch

In [55]:
for synset in synsets :
    print(synset.name())
    print("- POS : ", synset.lexname())
    print("- Definition : ", synset.definition())
    print("- Lemmas : ", synset.lemma_names())
    print()

present.n.01
- POS :  noun.time
- Definition :  the period of time that is happening now; any continuous stretch of time including the moment of speech
- Lemmas :  ['present', 'nowadays']

present.n.02
- POS :  noun.possession
- Definition :  something presented as a gift
- Lemmas :  ['present']

present.n.03
- POS :  noun.communication
- Definition :  a verb tense that expresses actions or states at the time of speaking
- Lemmas :  ['present', 'present_tense']

show.v.01
- POS :  verb.perception
- Definition :  give an exhibition of to an interested audience
- Lemmas :  ['show', 'demo', 'exhibit', 'present', 'demonstrate']

present.v.02
- POS :  verb.communication
- Definition :  bring forward and present to the mind
- Lemmas :  ['present', 'represent', 'lay_out']

stage.v.01
- POS :  verb.creation
- Definition :  perform (a play), especially on a stage
- Lemmas :  ['stage', 'present', 'represent']

present.v.04
- POS :  verb.possession
- Definition :  hand over formally
- Lemmas :  [

#### path_similarity()
WordNet은 어휘 간의 관계를 유사도로 나타낼 수 있음


In [56]:
# synset 객체를 단어별로 생성합니다. 
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree , lion , tiger , cat , dog]
similarities = []
entity_names = [ entity.name().split('.')[0] for entity in entities]

# 단어별 synset 들을 iteration 하면서 다른 단어들의 synset과 유사도를 측정합니다. 
for entity in entities:
    similarity = [ round(entity.path_similarity(compared_entity), 2)  for compared_entity in entities ]
    similarities.append(similarity)
    
# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame형태로 저장합니다.  
similarity_df = pd.DataFrame(similarities , columns=entity_names,index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


### 2) SentiWordNet
- WordNet의 Synset과 유사한 Senti_Synset 클래스 = Senti_Synset
- Synset과 유사하게 Senti_Synset 클래스를 리스트 형태로 반환

In [58]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type :', type(senti_synsets))
print('senti_synsets() 반환 값 갯수:', len(senti_synsets))
print('senti_synsets() 반환 값 :', senti_synsets)

senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 갯수: 11
senti_synsets() 반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


- SentiSynset 객체 = 감성 지수(긍정/부정) + 객관성 지수
- 어떤 단어가 전혀 감성적이지 않으면 객관성 지수는 1, 감성 지수는 모두 0

In [62]:
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score())
print('father 부정감성 지수: ', father.neg_score())
print('father 객관성 지수: ', father.obj_score())
print()
fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수: ',fabulous .pos_score())
print('fabulous 부정감성 지수: ',fabulous .neg_score())
print('fabulous 객관성 지수: ', fabulous.obj_score())


father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0

fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125
fabulous 객관성 지수:  0.0


### SentiWordNet을 이용한 영화 감상평 감성 분석

1. 문서(Document)를 문장 단위로 분해
2. 다시 문장을 단어(Word) 단위로 토근화하고 품사 태깅
3. 품사 태깅된 단어 기만으로 synset 객체와 senti_synset 객체를 생성
4. senti_synset에서 긍정 감성/부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정

In [63]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet기반의 품사 Tag로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return

- 문서를 문장 → 단어 토큰 → 품사 태깅 후에 SentiSynset 클래스를 생성하고 감성 지수를 합산하는 함수

In [64]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출  
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word , tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue                   
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산. 
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1
    
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0 :
        return 1
    
    return 0

In [65]:
review_df['preds'] = review_df['review'].apply( lambda x : swn_polarity(x) )
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

In [66]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score 
from sklearn.metrics import recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test=None, pred=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [67]:
print('#### SentiWordNet 예측 성능 평가 ####')
get_clf_eval(y_target, preds)

#### SentiWordNet 예측 성능 평가 ####
오차 행렬
[[7649 4851]
 [3578 8922]]
정확도: 0.6628, 정밀도: 0.6478, 재현율: 0.7138,    F1: 0.6792, AUC:0.6628


### 3) VADER
### VADER 를 이용한 감성 분석